In [1]:
!gdown 1mnimNISD3QGGGCcNJMHpBDDz9lWsmdsM
!unzip -q /kaggle/working/data.zip

Downloading...
From (original): https://drive.google.com/uc?id=1mnimNISD3QGGGCcNJMHpBDDz9lWsmdsM
From (redirected): https://drive.google.com/uc?id=1mnimNISD3QGGGCcNJMHpBDDz9lWsmdsM&confirm=t&uuid=7faab064-704b-4e70-8d6f-73835084a6bd
To: /kaggle/working/data.zip
100%|███████████████████████████████████████| 1.12G/1.12G [00:07<00:00, 158MB/s]


In [2]:
!pip install -q tensorflow_ranking javalang rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.15.1 which is incompatible.
jax 0.5.2 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.3.2 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but y

In [ ]:
import os
from pathlib import Path
import regex as re
import pandas as pd
import javalang
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import shap
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

from transformers import AutoTokenizer, AutoModel
import tensorflow_ranking as tfr
from collections import defaultdict
import gc
from tqdm import tqdm

2025-06-14 06:19:40.375391: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-14 06:19:40.375457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-14 06:19:40.377163: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [6]:
JAVA_KEYWORDS = ['abstract', 'assert', 'boolean', 'break', 'byte', 'case',
     'catch', 'char', 'class', 'const', 'continue', 'default', 'do', 'double',
     'else', 'enum', 'extends', 'false', 'final', 'finally', 'float', 'for', 'goto',
     'if', 'implements', 'import', 'instanceof', 'int', 'interface', 'long',
     'native', 'new', 'null', 'package', 'private', 'protected', 'public', 'return',
     'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this',
     'throw', 'throws', 'transient', 'true', 'try', 'void', 'volatile', 'while']
EMB_BATCH_SIZE = 64 #16
# original_content
MAX_CONTEXT_LENGTH = 512 #2048

MODEL_NAME = "microsoft/codebert-base" #'jinaai/jina-embeddings-v2-base-code'

ASPECTJ = {'repo_path':'data/aspectj', 'source_path':'data/org.aspectj-bug433351', 'report_path':'data/AspectJ.xlsx', 'name': 'aspectj'}
BIRT = {'repo_path':'', 'source_path':'data/birt-20140211-1400', 'report_path':'data/Birt.xlsx', 'name': 'birt'}
ECLIPSE = {'repo_path':'', 'source_path':'data/eclipse.platform.ui-johna-402445', 'report_path':'data/Eclipse_Platform_UI.xlsx', 'name': 'eclipse'}
SWT = {'repo_path':'', 'source_path':'data/eclipse.platform.swt-xulrunner-31', 'report_path':'data/SWT.xlsx', 'name': 'swt'}
TOMCAT = {'repo_path':'data/tomcat', 'source_path':'data/tomcat-7.0.51', 'report_path':'data/Tomcat.xlsx', 'name': 'tomcat'}

In [7]:
stop_words_english = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text, is_code=False):
    if not isinstance(text, str):
        text = ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    processed_tokens = []
    if is_code:
        for token in tokens:
            parts = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', token)).split()
            parts = [p for sub_part in parts for p in sub_part.split('_') if p]
            processed_tokens.extend([stemmer.stem(lemmatizer.lemmatize(part)) for part in parts if part not in JAVA_KEYWORDS and part not in stop_words_english and len(part)>1])
    else:
        processed_tokens = [stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens if token not in stop_words_english and len(token)>1]
    return " ".join(processed_tokens)

In [ ]:
def get_params_type_str(params):
    if params is None:
        return '()'
    raw_names = []
    for param in params:
        if param.type is None:
            continue
        name_parts = []
        current_node = param.type
        while current_node:
            name_parts.append(current_node.name)
            current_node = getattr(current_node, 'sub_type', None)
        raw_name = '.'.join(name_parts)
        raw_name += '[]' * (len(param.type.dimensions) + (1 if param.varargs else 0))
        raw_names.append(raw_name)
    return '(' + ', '.join(raw_names) + ')'

def generate_java_file_summary(file_content_str):
    summary_parts = []
    identifiers = set()
    try:
        tree = javalang.parse.parse(file_content_str)
        
        for _, node in tree.filter(javalang.tree.Documented):
             if node.documentation:
                summary_parts.append(f"javadoc: {preprocess_text(node.documentation, is_code=True)}")

        for _, node in tree.filter(javalang.tree.TypeDeclaration):
            summary_parts.append(f"type_declaration: {node.name}")
            identifiers.add(node.name)
            if hasattr(node, 'extends') and node.extends:
                if isinstance(node.extends, list):
                    for ext in node.extends:
                        if hasattr(ext, 'name'):
                            summary_parts.append(f"extends: {ext.name}")
                            identifiers.add(ext.name)
                elif hasattr(node.extends, 'name'):
                    summary_parts.append(f"extends: {node.extends.name}")
                    identifiers.add(node.extends.name)
            if hasattr(node, 'implements') and node.implements:
                 for impl in node.implements:
                     if hasattr(impl,'name'):
                        summary_parts.append(f"implements: {impl.name}")
                        identifiers.add(impl.name)
        
        for _, node in tree.filter(javalang.tree.MethodDeclaration):
            summary_parts.append(f"method: {node.name}{get_params_type_str(node.parameters)}")
            identifiers.add(node.name)
        for _, node in tree.filter(javalang.tree.ConstructorDeclaration):
            summary_parts.append(f"constructor: {node.name}{get_params_type_str(node.parameters)}")
            identifiers.add(node.name)
        
        for _, node in tree.filter(javalang.tree.FieldDeclaration):
            field_type = node.type.name if node.type and hasattr(node.type, 'name') else "unknown_type"
            for declarator in node.declarators:
                 if hasattr(declarator, 'name'):
                    summary_parts.append(f"field: {field_type} {declarator.name}")
                    identifiers.add(declarator.name)
        
        for _, node in tree.filter(javalang.tree.EnumConstantDeclaration):
            summary_parts.append(f"enum_constant: {node.name}")
            identifiers.add(node.name)

    except (javalang.parser.JavaSyntaxError, javalang.tokenizer.LexerError, TypeError, AttributeError, StopIteration) as e:
        summary_parts.append(f"raw_code_snippet: {file_content_str}")

    return " ".join(filter(None, summary_parts)), identifiers

In [ ]:
class BugReport:
    def __init__(self, bug_id, text, relevant_file_paths, project_name):
        self.bug_id = bug_id
        self.project_name = project_name
        self.original_text = text
        self.processed_text = preprocess_text(text, is_code=False)
        self.relevant_file_paths = relevant_file_paths
        self.mentioned_simple_class_names = self._extract_mentioned_simple_class_names(self.original_text)
        self.stack_trace_files = self._extract_files_from_stack_trace(self.original_text)
        self.mentioned_identifiers = self._extract_identifiers_from_text(self.processed_text)

    def _extract_mentioned_simple_class_names(self, text):
        if not isinstance(text, str):
            return set()
        try:
            qualified_names_matches = re.findall(r'\b([a-z_][a-z0-9_]*(?:\.[a-z_][a-z0-9_]*)*\.)([A-Z][a-zA-Z0-9_]*)\b', text)
            standalone_or_java_file_matches = re.findall(r'(?<![./\w])\b([A-Z][a-zA-Z0-9_]*)(\.java)?\b', text)
        except Exception as e:
            qualified_names_matches = []
            standalone_or_java_file_matches = []
        
        normalized_class_names = set()
        for match in qualified_names_matches:
            normalized_class_names.add(match[1].lower())
        for match in standalone_or_java_file_matches:
            normalized_class_names.add(match[0].lower())
        return normalized_class_names

    def _extract_files_from_stack_trace(self, text):
        if not isinstance(text, str):
            return set()
        matches = re.findall(r'\((([A-Z][a-zA-Z0-9_]*)\.java):\d+\)', text)
        return {match[0] for match in matches} # Returns a set of filenames

    def _extract_identifiers_from_text(self, processed_text):
        return set(re.findall(r'\b[a-z]+[A-Z][a-zA-Z0-9_]*\b', processed_text))


class SourceJavaFile:
    def __init__(self, id, file_path, original_content, summary_text, project_name, identifiers):
        self.id = id
        self.project_name = project_name
        self.file_path = file_path
        self.original_content = original_content
        self.raw_summary = summary_text
        self.processed_summary = preprocess_text(summary_text, is_code=True)
        self.processed_original_content = preprocess_text(original_content, is_code=True)
        self.identifiers = identifiers


def load_dataset_file_level(project_config, all_projects_source_files_map, all_projects_bug_reports_map):
    print(f"Loading dataset for: {project_config['name']}")
    source_path = Path(project_config['source_path'])
    report_path = project_config['report_path']

    project_source_files = []
    java_files_paths = sorted(source_path.rglob("*.java"), key=lambda x: x.as_posix())
    print(f"Found {len(java_files_paths)} Java files for {project_config['name']}.")

    for file_p in java_files_paths:
        try:
            with open(file_p, "r", encoding="latin-1") as f:
                file_content = f.read()
            relative_file_path = file_p.as_posix().replace(source_path.as_posix() + "/", "")
            file_summary_str, file_identifiers = generate_java_file_summary(file_content)
            
            sjf = SourceJavaFile(
                id=relative_file_path,
                file_path=relative_file_path,
                original_content=file_content,
                summary_text=file_summary_str,
                project_name=project_config['name'],
                identifiers=file_identifiers
            )
            project_source_files.append(sjf)
        except Exception as e:
            print(f"Error processing file {file_p} for summary: {e}")
    
    all_projects_source_files_map[project_config['name']] = project_source_files
    print(f"Processed {len(project_source_files)} source Java files for {project_config['name']}.")

    reports_df = pd.read_excel(report_path)
    reports_df.sort_values(by=['report_timestamp'], inplace=True)
    reports_df['text'] = (reports_df['summary'] + ' ' + reports_df['description'].fillna(''))

    def _clean_br_text(text):
        text = re.sub(r'[ \t]+$|^Bug \d+\s*', '', text, flags=re.MULTILINE)
        return re.sub(r'\n\s*\n+', '\n\n', text)
    reports_df['text'] = reports_df['text'].apply(_clean_br_text)
    
    project_bug_reports = []
    for idx, row in reports_df.iterrows():
        bug_id = f"{project_config['name']}_bug_{row.get('bug_id', idx)}"
        file_paths_str = row['files']
        chunks_paths = file_paths_str.strip().split('.java ')
        fixed_relevant_file_paths = set()
        for i, chunk_path_part in enumerate(chunks_paths):
            path = chunk_path_part.strip()
            if i != len(chunks_paths) - 1:
                path += '.java'
            fixed_relevant_file_paths.add(path)

        actual_relevant_paths_in_source = set()
        parsed_file_paths_in_project = {sjf.file_path for sjf in project_source_files}

        for reported_path in fixed_relevant_file_paths:
            if reported_path in parsed_file_paths_in_project:
                 actual_relevant_paths_in_source.add(reported_path)

        if actual_relevant_paths_in_source:
            br = BugReport(bug_id, row['text'], actual_relevant_paths_in_source, project_config['name'])
            project_bug_reports.append(br)
    
    all_projects_bug_reports_map[project_config['name']] = project_bug_reports
    print(f"Processed {len(project_bug_reports)} bug reports for {project_config['name']}.")

In [ ]:
def calculate_tfidf_scores(bug_report_processed_text, all_file_processed_summaries, target_file_processed_summary):
    if not bug_report_processed_text or not target_file_processed_summary:
        return 0.0
    vectorizer = TfidfVectorizer(analyzer='word', stop_words=JAVA_KEYWORDS, min_df=2, max_df=0.95, ngram_range=(1,2))
    corpus = [bug_report_processed_text] + all_file_processed_summaries # Corpus for this bug vs all files
    try:
        tfidf_matrix = vectorizer.fit_transform(corpus)
        bug_vec = tfidf_matrix[0]
        target_idx = all_file_processed_summaries.index(target_file_processed_summary) + 1
        if target_idx >= tfidf_matrix.shape[0]:
            return 0.0
        file_vec = tfidf_matrix[target_idx]
        return float(cosine_similarity(bug_vec, file_vec)[0][0])
    except (ValueError, IndexError):
        print('calculate_tfidf_scores')
        return 0.0

def calculate_bm25_scores(bug_report_processed_text, all_file_processed_summaries, target_file_processed_summary):
    if not bug_report_processed_text or not target_file_processed_summary:
        return 0.0
    tokenized_query = bug_report_processed_text.split()
    tokenized_corpus = [doc.split() for doc in all_file_processed_summaries]
    if not tokenized_corpus or not any(tokenized_corpus) or not tokenized_query:
        return 0.0
    try:
        bm25 = BM25Okapi(tokenized_corpus)
        target_doc_idx = all_file_processed_summaries.index(target_file_processed_summary)
        all_doc_scores = bm25.get_scores(tokenized_query)
        if target_doc_idx < len(all_doc_scores):
            return float(all_doc_scores[target_doc_idx])
        return 0.0
    except (ValueError, IndexError):
        print('calculate_bm25_scores')
        return 0.0

def calculate_exact_simple_classname_match_score(bug_report_mentioned_simple_class_names, source_file_full_path_str):
    # 1 if the source file's class name is mentioned in the bug report. 0 else.
    if not bug_report_mentioned_simple_class_names or not source_file_full_path_str:
        return 0.0
    try:
        source_filename = Path(source_file_full_path_str).name
        source_simple_classname_normalized = Path(source_filename).stem.lower()
    except Exception:
        print('calculate_exact_simple_classname_match_score')
        return 0.0

    if source_simple_classname_normalized in bug_report_mentioned_simple_class_names:
        return 1.0
    
    return 0.0

In [ ]:
def calculate_stack_trace_match_score(bug_report_stack_trace_files, source_file_full_path_str):
    if not bug_report_stack_trace_files or not source_file_full_path_str:
        return 0.0
    
    source_filename = Path(source_file_full_path_str).name
    return 1.0 if source_filename in bug_report_stack_trace_files else 0.0

def calculate_identifier_match_score(bug_report_identifiers, source_file_identifiers):
    if not bug_report_identifiers or not source_file_identifiers:
        return 0.0
    
    intersection = len(bug_report_identifiers.intersection(source_file_identifiers))
    union = len(bug_report_identifiers.union(source_file_identifiers))
    
    return float(intersection / union) if union > 0 else 0.0

In [12]:
def get_embedding_model_and_tokenizer(model_name=MODEL_NAME):
    print(f"Loading pre-trained model ({model_name}) for embeddings...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
    model.eval()
    return model, tokenizer

In [13]:
def generate_embeddings(model, tokenizer, texts, device, batch_size=EMB_BATCH_SIZE, max_len=MAX_CONTEXT_LENGTH):
    model.to(device)
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        if not batch_texts or all(not t for t in batch_texts):
            emb_dim = model.config.hidden_size if hasattr(model.config, 'hidden_size') else 768
            all_embeddings.extend([torch.zeros(emb_dim).cpu()] * len(batch_texts))
            continue
        inputs = tokenizer(batch_texts, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :] 
        all_embeddings.extend(cls_embeddings.cpu())
    return torch.stack(all_embeddings) if all_embeddings else torch.empty(0)

def calculate_s_emb_model_cosine_similarity(emb1, emb2):
    if emb1 is None or emb2 is None or emb1.nelement() == 0 or emb2.nelement() == 0:
        return 0.0
    if emb1.ndim == 1:
        emb1 = emb1.unsqueeze(0)
    if emb2.ndim == 1:
        emb2 = emb2.unsqueeze(0)

    emb1_cpu = emb1.cpu().numpy()
    emb2_cpu = emb2.cpu().numpy()
    return float(cosine_similarity(emb1_cpu, emb2_cpu)[0][0])

In [14]:
class CombinerNN(nn.Module):
    def __init__(self, input_dim=4, hidden_dim1=300, hidden_dim2=150, output_dim=1):
        super(CombinerNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        return self.fc3(x)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.999, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, inputs, targets): # inputs are logits
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets.float(), reduction='none')
        pt = torch.exp(-BCE_loss)
        alpha_t = torch.where(targets.float() == 1, self.alpha, 1 - self.alpha) # alpha for positive, 1-alpha for negative
        focal_loss = alpha_t * (1 - pt) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        return focal_loss.sum() if self.reduction == 'sum' else focal_loss

In [ ]:
#FAILED
def bootstrapped_batch_generator(train_data, batch_size, epoch_num):
    positive_samples = [d for d in train_data if d['label'] == 1]
    negative_samples = [d for d in train_data if d['label'] == 0]

    num_pos_per_batch = batch_size // 2
    num_neg_per_batch = batch_size - num_pos_per_batch

    rng = np.random.default_rng(seed=epoch_num)
    rng.shuffle(negative_samples)

    num_batches = len(negative_samples) // num_neg_per_batch

    for i in range(num_batches):
        # get negative samples without replacement
        start_neg_idx = i * num_neg_per_batch
        end_neg_idx = start_neg_idx + num_neg_per_batch
        neg_batch_samples = negative_samples[start_neg_idx:end_neg_idx]

        # randomly pick positive instances with replacement (bootstrapping)
        pos_indices = rng.choice(len(positive_samples), size=num_pos_per_batch, replace=True)
        pos_batch_samples = [positive_samples[j] for j in pos_indices]

        # combine, then shuffle
        combined_batch = pos_batch_samples + neg_batch_samples
        rng.shuffle(combined_batch)

        # extract features
        features = np.array([[d['s_tfidf'], d['s_bm25'], d['s_emb_model'], d['s_filepath'], d['s_stack_trace'], d['s_identifier']] for d in combined_batch], dtype=np.float32)
        labels = np.array([d['label'] for d in combined_batch], dtype=np.float32)

        yield (torch.tensor(features), torch.tensor(labels).unsqueeze(1))

In [ ]:
# FAILED
MU_DAYS = 365

def calculate_fixing_recency(bug_report, source_file_path, file_commit_history):
    report_ts = bug_report.report_timestamp
    commit_timestamps = file_commit_history.get(source_file_path, [])
    if not commit_timestamps:
        return 0.0
    mu_timedelta = pd.to_timedelta(MU_DAYS, unit='d').total_seconds()
    
    recent_commits = [ts for ts in commit_timestamps if (report_ts > ts) and (report_ts - ts <= mu_timedelta)]
    
    if not recent_commits:
        return 0.0

    most_recent_commit_ts = max(recent_commits)
    elapsed_seconds = report_ts - most_recent_commit_ts
    elapsed_days = elapsed_seconds / (24 * 3600)
    if elapsed_days < 1:
        elapsed_days = 1
        
    return 1.0 / elapsed_days

def calculate_fixing_frequency(bug_report, source_file_path, file_commit_history):
    report_ts = bug_report.report_timestamp

    commit_timestamps = file_commit_history.get(source_file_path, [])
    if not commit_timestamps:
        return 0

    mu_timedelta = pd.to_timedelta(MU_DAYS, unit='d').total_seconds()
    
    recent_commits_count = sum(1 for ts in commit_timestamps if (report_ts > ts) and (report_ts - ts <= mu_timedelta))
    
    return float(recent_commits_count)

In [ ]:
def train_combiner_nn(model, train_features_tensor, train_labels_tensor, num_epochs=20, lr=1e-4, batch_size=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = FocalLoss().to(device)
    train_dataset = torch.utils.data.TensorDataset(train_features_tensor, train_labels_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    model.train()
    avg_loss = 0
    for epoch in tqdm(range(num_epochs)):
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs_logits = model(features)
            loss = criterion(outputs_logits.squeeze(), labels.squeeze())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
    avg_loss /= num_epochs
    return model, avg_loss

def kfold_split_indices(data_size, k):
    indices = np.arange(data_size)
    folds = []
    fold_size = data_size // k
    for i in range(k):
        start = i * fold_size
        # The last fold should take all remaining indices
        end = (i + 1) * fold_size if i < k - 1 else data_size
        folds.append(indices[start:end])
    return folds


def train_and_evaluate_file_level_localization(project_name_to_load, k_folds=10, num_sample_neg=None, num_train_fold=3):
    all_source_files_map, all_bug_reports_map = {}, {}
    project_configs = [ASPECTJ, BIRT, ECLIPSE, SWT, TOMCAT]
    target_config = next((p for p in project_configs if p['name'] == project_name_to_load), None)
    if not target_config:
        print(f"Config for {project_name_to_load} not found.")
        return
    
    load_dataset_file_level(target_config, all_source_files_map, all_bug_reports_map)
    project_bug_reports = all_bug_reports_map.get(project_name_to_load, [])
    project_source_files = all_source_files_map.get(project_name_to_load, [])
    if not project_bug_reports or not project_source_files:
        print(f"No data for {project_name_to_load}.")
        return

    fold_br_indices = kfold_split_indices(len(project_bug_reports), k_folds)
    all_fold_metrics_results = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print("Loading embedding model & pre-computing ALL embeddings (once)...")
    emb_model, cb_tokenizer = get_embedding_model_and_tokenizer()
    br_embeddings_map = {br.bug_id: emb for br, emb in zip(project_bug_reports, 
                        generate_embeddings(emb_model, cb_tokenizer, 
                                            [br.original_text for br in project_bug_reports], device))}
    sf_embeddings_map = {sf.id: emb for sf, emb in zip(project_source_files, 
                        generate_embeddings(emb_model, cb_tokenizer, 
                                            [sf.raw_summary for sf in project_source_files], device))}
    del emb_model, cb_tokenizer; gc.collect(); torch.cuda.empty_cache() if torch.cuda.is_available() else None

    combiner_nn = CombinerNN(input_dim=6)
    
    for i_fold in range(k_folds):
        print(f"\n--- Fold {i_fold + 1}/{k_folds} ---")
        train_br_fold_indices = fold_br_indices[i_fold]
        test_br_fold_indices = fold_br_indices[(i_fold + 1) % k_folds]
        train_brs = [project_bug_reports[idx] for idx in train_br_fold_indices]
        test_brs = [project_bug_reports[idx] for idx in test_br_fold_indices]
        if not train_brs or not test_brs:
            print("Skipping fold due to empty train/test BRs.")
            continue

        print("Pre-fitting lexical models for the fold...")
        all_sf_summaries_processed_fold = [sf.processed_original_content for sf in project_source_files]
        
        fold_tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words=JAVA_KEYWORDS, min_df=2, max_df=0.95, ngram_range=(1,2))
        fold_file_tfidf_matrix = None
        if all_sf_summaries_processed_fold and not all(not s for s in all_sf_summaries_processed_fold):
            try:
                fold_file_tfidf_matrix = fold_tfidf_vectorizer.fit_transform(all_sf_summaries_processed_fold)
            except ValueError:
                print("TF-IDF fit error (e.g. empty vocab). TF-IDF scores will be 0.")
        else:
            print("Empty corpus for TF-IDF. Scores will be 0.")

        tokenized_all_files_bm25_fold = [s.split() for s in all_sf_summaries_processed_fold]
        fold_bm25_model = None
        if tokenized_all_files_bm25_fold and not all(not t for t in tokenized_all_files_bm25_fold):
            try:
                fold_bm25_model = BM25Okapi(tokenized_all_files_bm25_fold)
            except Exception:
                print("BM25 init error. Scores will be 0.")
        else:
            print("Empty corpus for BM25. Scores will be 0.")
        
        train_fold_data_for_combiner, test_fold_data_for_combiner_features = [], []
        test_items_for_eval_metadata = []

        print(f"Preparing training data (Fold {i_fold+1})...")
        negative_train_pairs_temp = []

        for br in train_brs:
            br_emb = br_embeddings_map.get(br.bug_id); br_proc_txt = br.processed_text
            if br_emb is None:
                continue

            s_tfidf_vec, s_bm25_vec = np.zeros(len(project_source_files)), np.zeros(len(project_source_files))
            if fold_file_tfidf_matrix is not None and br_proc_txt:
                try:
                    s_tfidf_vec = cosine_similarity(fold_tfidf_vectorizer.transform([br_proc_txt]), fold_file_tfidf_matrix)[0]
                except ValueError:
                    pass
            if fold_bm25_model is not None and br_proc_txt:
                tok_br_txt = br_proc_txt.split()
                if tok_br_txt: 
                    try:
                        s_bm25_vec = fold_bm25_model.get_scores(tok_br_txt)
                    except Exception:
                        pass
            
            for sf_idx, sf in enumerate(project_source_files):
                sf_emb = sf_embeddings_map.get(sf.id)
                if sf_emb is None:
                    continue

                features = {
                    "s_tfidf": s_tfidf_vec[sf_idx], 
                    "s_bm25": s_bm25_vec[sf_idx], 
                    "s_emb_model": calculate_s_emb_model_cosine_similarity(br_emb, sf_emb),
                    "s_filepath": calculate_exact_simple_classname_match_score(br.mentioned_simple_class_names, sf.file_path),
                    "s_stack_trace": calculate_stack_trace_match_score(br.stack_trace_files, sf.file_path),
                    "s_identifier": calculate_identifier_match_score(br.mentioned_identifiers, sf.identifiers)
                }
                label = 1 if sf.file_path in br.relevant_file_paths else 0
                
                if label == 1:
                    train_fold_data_for_combiner.append({**features, "label": label})
                elif num_sample_neg is not None:
                    negative_train_pairs_temp.append({**features, "label": label})
                else:
                    train_fold_data_for_combiner.append({**features, "label": label})
        
        if num_sample_neg is not None and negative_train_pairs_temp:
            num_neg_to_sample = num_sample_neg
            if num_neg_to_sample < len(negative_train_pairs_temp):
                sampled_indices = np.random.choice(len(negative_train_pairs_temp), num_neg_to_sample, replace=False)
                train_fold_data_for_combiner.extend([negative_train_pairs_temp[i] for i in sampled_indices])
            else:
                train_fold_data_for_combiner.extend(negative_train_pairs_temp)
        print(f"Total training pairs for Combiner NN: {len(train_fold_data_for_combiner)}")


        print(f"Preparing testing data (Fold {i_fold+1})...")
        for br in test_brs:
            br_emb = br_embeddings_map.get(br.bug_id); br_proc_txt = br.processed_text
            if br_emb is None:
                continue
            
            s_tfidf_vec, s_bm25_vec = np.zeros(len(project_source_files)), np.zeros(len(project_source_files))
            if fold_file_tfidf_matrix is not None and br_proc_txt:
                try:
                    s_tfidf_vec = cosine_similarity(fold_tfidf_vectorizer.transform([br_proc_txt]), fold_file_tfidf_matrix)[0]
                except ValueError:
                    pass
            if fold_bm25_model is not None and br_proc_txt:
                tok_br_txt = br_proc_txt.split()
                if tok_br_txt: 
                    try:
                        s_bm25_vec = fold_bm25_model.get_scores(tok_br_txt)
                    except Exception:
                        pass
            
            for sf_idx, sf in enumerate(project_source_files):
                sf_emb = sf_embeddings_map.get(sf.id)
                if sf_emb is None:
                    continue

                features = {
                    "s_tfidf": s_tfidf_vec[sf_idx], 
                    "s_bm25": s_bm25_vec[sf_idx], 
                    "s_emb_model": calculate_s_emb_model_cosine_similarity(br_emb, sf_emb),
                    "s_filepath": calculate_exact_simple_classname_match_score(br.mentioned_simple_class_names, sf.file_path),
                    "s_stack_trace": calculate_stack_trace_match_score(br.stack_trace_files, sf.file_path),
                    "s_identifier": calculate_identifier_match_score(br.mentioned_identifiers, sf.identifiers)
                }
                test_fold_data_for_combiner_features.append(features)
                label = 1 if sf.file_path in br.relevant_file_paths else 0
                test_items_for_eval_metadata.append({'bug_id': br.bug_id, 'file_path': sf.file_path, 'true_label': label})

        if not train_fold_data_for_combiner:
            print(f"No training data for Combiner in fold {i_fold+1}.")
            continue
        if i_fold < num_train_fold:
            print("Training Combiner NN...")
            train_features = np.array([[d['s_tfidf'], d['s_bm25'], d['s_emb_model'], d['s_filepath'], d['s_stack_trace'], d['s_identifier']] for d in train_fold_data_for_combiner], dtype=np.float32)
            train_labels = np.array([d['label'] for d in train_fold_data_for_combiner], dtype=np.float32)
            
            combiner_nn, avg_loss = train_combiner_nn(combiner_nn, torch.tensor(train_features), 
                                            torch.tensor(train_labels).unsqueeze(1), batch_size=128)
    
            if not test_fold_data_for_combiner_features:
                print(f"No test data for Combiner in fold {i_fold+1}.")
                continue
            print("Train loss: {:.3f}".format(avg_loss))
            
        print("Evaluating Combiner NN...")
        combiner_nn.eval().to(device)
        test_features_np = np.array([[d['s_tfidf'], d['s_bm25'], d['s_emb_model'], d['s_filepath'], d['s_stack_trace'], d['s_identifier']] for d in test_fold_data_for_combiner_features], dtype=np.float32)

        final_scores_for_test = []
        with torch.no_grad():
            test_loader = torch.utils.data.DataLoader(torch.tensor(test_features_np), batch_size=512)
            for batch_feats in test_loader:
                outputs_logits = combiner_nn(batch_feats.to(device))
                final_scores_for_test.extend(torch.sigmoid(outputs_logits).squeeze().cpu().numpy())
        
        for i, score in enumerate(final_scores_for_test):
            test_items_for_eval_metadata[i]['predicted_score'] = score

        scores_by_bug, labels_by_bug = defaultdict(list), defaultdict(list)
        for item in test_items_for_eval_metadata:
            scores_by_bug[item['bug_id']].append(item['predicted_score'])
            labels_by_bug[item['bug_id']].append(item['true_label'])
        
        k_vals = [1, 2, 3, 4, 5, 10, 15]; topk_s = {k: [] for k in k_vals}; mrr_s, map_s = [], []
        for bug_id in scores_by_bug:
            pred_s_np = np.array(scores_by_bug[bug_id], dtype=np.float32)
            true_l_np = np.array(labels_by_bug[bug_id], dtype=np.float32)
            if len(pred_s_np) == 0:
                continue
            s_indices = np.argsort(-pred_s_np)
            y_true_s = true_l_np[s_indices].reshape(1, -1); y_pred_s = pred_s_np[s_indices].reshape(1, -1)
            for k_v in k_vals: topk_s[k_v].append(tfr.keras.metrics.HitsMetric(topn=k_v)(y_true_s, y_pred_s).numpy())
            mrr_s.append(tfr.keras.metrics.MRRMetric()(y_true_s, y_pred_s).numpy())
            map_s.append(tfr.keras.metrics.MeanAveragePrecisionMetric()(y_true_s, y_pred_s).numpy())

        current_fold_eval = {f"file_top_{k_v}_acc": np.mean(topk_s[k_v]) if topk_s[k_v] else 0.0 for k_v in k_vals}
        current_fold_eval["file_mrr"] = np.mean(mrr_s) if mrr_s else 0.0
        current_fold_eval["file_map"] = np.mean(map_s) if map_s else 0.0
        print(f"Fold {i_fold + 1} Eval: {current_fold_eval}"); all_fold_metrics_results.append(current_fold_eval)
        
        if i_fold == k_folds - 1:
            try:
                explainer = shap.Explainer(combiner_nn, torch.tensor(test_features_np[:100]).to(device))
                shap_values = explainer(torch.tensor(test_features_np[:100]).to(device))
                feature_names = ["TF-IDF", "BM25", "Embedding Sim.", "Filepath Match", "Stack Trace", "Identifier Match"]
                shap.summary_plot(shap_values, features=test_features_np[:100], feature_names=feature_names)
            except Exception as e:
                print(f"Could not generate SHAP plot: {e}")

        gc.collect()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    if all_fold_metrics_results:
        avg_metrics = {k: np.mean([f[k] for f in all_fold_metrics_results if k in f]) for k in all_fold_metrics_results[0]}
        print("\n--- Average Metrics ---"); [print(f"{k}: {v:.4f}") for k, v in avg_metrics.items()]
    else:
        print("No metrics collected.")
    
    return all_fold_metrics_results

In [17]:
project_name = TOMCAT['name'] 
all_fold_metrics_results = train_and_evaluate_file_level_localization(project_name, k_folds=10, num_sample_neg=None, num_train_fold=2)
pd.DataFrame(all_fold_metrics_results).to_csv('all_fold_metrics_results.csv')

Loading dataset for: tomcat
Found 1794 Java files for tomcat.
Processed 1794 source Java files for tomcat.
Processed 1016 bug reports for tomcat.
Loading embedding model & pre-computing ALL embeddings (once)...
Loading pre-trained model (microsoft/codebert-base) for embeddings...

--- Fold 1/10 ---
Pre-fitting lexical models for the fold...
Preparing training data (Fold 1)...
Total training pairs for Combiner NN: 181194
Preparing testing data (Fold 1)...
Training Combiner NN...


100%|██████████| 20/20 [01:24<00:00,  4.21s/it]


Train loss: 0.694
Evaluating Combiner NN...
Fold 1 Eval: {'file_top_1_acc': 0.48514852, 'file_top_2_acc': 0.57425743, 'file_top_3_acc': 0.63366336, 'file_top_4_acc': 0.6831683, 'file_top_5_acc': 0.6930693, 'file_top_10_acc': 0.76237625, 'file_top_15_acc': 0.7920792, 'file_mrr': 0.5809696, 'file_map': 0.5553111}

--- Fold 2/10 ---
Pre-fitting lexical models for the fold...
Preparing training data (Fold 2)...
Total training pairs for Combiner NN: 181194
Preparing testing data (Fold 2)...
Training Combiner NN...


100%|██████████| 20/20 [01:24<00:00,  4.21s/it]


Train loss: 0.372
Evaluating Combiner NN...
Fold 2 Eval: {'file_top_1_acc': 0.46534654, 'file_top_2_acc': 0.5445545, 'file_top_3_acc': 0.57425743, 'file_top_4_acc': 0.5940594, 'file_top_5_acc': 0.63366336, 'file_top_10_acc': 0.7128713, 'file_top_15_acc': 0.7425743, 'file_mrr': 0.5454886, 'file_map': 0.50811124}

--- Fold 3/10 ---
Pre-fitting lexical models for the fold...
Preparing training data (Fold 3)...
Total training pairs for Combiner NN: 181194
Preparing testing data (Fold 3)...
Evaluating Combiner NN...
Fold 3 Eval: {'file_top_1_acc': 0.4950495, 'file_top_2_acc': 0.55445546, 'file_top_3_acc': 0.64356434, 'file_top_4_acc': 0.65346533, 'file_top_5_acc': 0.6633663, 'file_top_10_acc': 0.7128713, 'file_top_15_acc': 0.75247526, 'file_mrr': 0.57116747, 'file_map': 0.48506972}

--- Fold 4/10 ---
Pre-fitting lexical models for the fold...
Preparing training data (Fold 4)...
Total training pairs for Combiner NN: 181194
Preparing testing data (Fold 4)...
Evaluating Combiner NN...
Fold 4 E